In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy.spatial import Voronoi, voronoi_plot_2d
from shapely.geometry import Polygon, Point
from shapely.ops import unary_union
import mplcursors

In [ ]:
!pip install mplcursors

In [110]:
players = pd.read_csv("players.csv")
plays = pd.read_csv("plays.csv")
playerplays = pd.read_csv("player_play.csv")
df_tracking= pd.read_csv('tracking_week_2.csv')

In [111]:
test = df_tracking.merge(players, on="nflId")

samplePlay = test[
    (test["gameId"] == 2022091805) &
    (test["playId"] == 2207) &
    (test["position"].isin(['WR', 'TE', 'RB']))
]

samplePlay.head(50)

,gameId,playId,nflId,displayName_x,frameId,frameType,time,jerseyNumber,club,playDirection,...,dis,o,dir,event,height,weight,birthDate,collegeName,position,displayName_y
3713900,2022091805,2207,43329.0,Sterling Shepard,1,BEFORE_SNAP,2022-09-18 18:49:34.6,3.0,NYG,left,...,0.02,263.29,214.10,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713901,2022091805,2207,43329.0,Sterling Shepard,2,BEFORE_SNAP,2022-09-18 18:49:34.7,3.0,NYG,left,...,0.00,270.39,12.80,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713902,2022091805,2207,43329.0,Sterling Shepard,3,BEFORE_SNAP,2022-09-18 18:49:34.8,3.0,NYG,left,...,0.02,276.62,6.17,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713903,2022091805,2207,43329.0,Sterling Shepard,4,BEFORE_SNAP,2022-09-18 18:49:34.9,3.0,NYG,left,...,0.05,279.39,3.09,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713904,2022091805,2207,43329.0,Sterling Shepard,5,BEFORE_SNAP,2022-09-18 18:49:35,3.0,NYG,left,...,0.07,282.20,359.82,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713905,2022091805,2207,43329.0,Sterling Shepard,6,BEFORE_SNAP,2022-09-18 18:49:35.1,3.0,NYG,left,...,0.10,284.18,358.72,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713906,2022091805,2207,43329.0,Sterling Shepard,7,BEFORE_SNAP,2022-09-18 18:49:35.2,3.0,NYG,left,...,0.14,288.64,358.17,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713907,2022091805,2207,43329.0,Sterling Shepard,8,BEFORE_SNAP,2022-09-18 18:49:35.3,3.0,NYG,left,...,0.17,292.58,358.03,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713908,2022091805,2207,43329.0,Sterling Shepard,9,BEFORE_SNAP,2022-09-18 18:49:35.4,3.0,NYG,left,...,0.20,296.14,357.65,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard
3713909,2022091805,2207,43329.0,Sterling Shepard,10,BEFORE_SNAP,2022-09-18 18:49:35.5,3.0,NYG,left,...,0.22,297.79,356.92,NaN,5-10,201,1993-02-10,Oklahoma,WR,Sterling Shepard


In [112]:
club_colors = {
    'ARI': '#97233F', 'ATL': '#000000', 'BAL': '#241773',
    'BUF': '#00338D', 'CAR': '#0085CA', 'CHI': '#C83803',
    'CIN': '#FB4F14', 'CLE': '#311D00', 'DAL': '#003594',
    'DEN': '#FB4F14', 'DET': '#FB4F14', 'GB': '#203731',
    'HOU': '#03202F', 'IND': '#002C5F', 'JAX': '#101820',
    'KC': '#E31837', 'LV': '#A5ACAF', 'LAC': '#0080C6',
    'LAR': '#003594', 'MIA': '#008E97', 'MIN': '#4F2683',
    'NE': '#002244', 'NO': '#D3BC8D', 'NYG': '#0B2265',
    'NYJ': '#125740', 'PHI': '#004C54', 'PIT': '#FFB612',
    'SF': '#AA0000', 'SEA': '#002244', 'TB': '#D50A0A',
    'TEN': '#0C2340', 'WAS': '#5A1414'
}

In [113]:
def compute_voronoi_areas(samplePlay):
    # Merge play details
    samplePlay = pd.merge(
        samplePlay,
        plays[['gameId', 'playId', 'possessionTeam', 'yardlineSide', 'yardlineNumber', 'absoluteYardlineNumber']],
        how='left',
        on=['gameId', 'playId']
    )
    
    # Extract relevant columns
    player_points = samplePlay[['nflId', 'x', 'y', 'position', 'frameId', 'playId','displayName_x']]
    # Get player coordinates
    points = player_points[['x', 'y']].values

    # Define field boundaries
    x_min, x_max = 0, 120
    y_min, y_max = 0, 53.3
    boundary_polygon = Polygon([(x_min, y_min), (x_min, y_max), (x_max, y_max), (x_max, y_min)])

    # Add boundary points to ensure Voronoi diagram is bounded
    boundary_points = np.array([[x_min, y_min], [x_min, y_max], [x_max, y_min], [x_max, y_max]])
    all_points = np.vstack([points, boundary_points])

    # Compute Voronoi diagram
    vor = Voronoi(all_points)

    # Clip Voronoi regions to field boundary
    clipped_areas = []
    for region, player_row in zip(vor.point_region[:-4], player_points.itertuples(index=False)):
        region_vertices = vor.regions[region]
        if -1 in region_vertices or not region_vertices:
            continue  # Skip unbounded or empty regions
        polygon = Polygon([vor.vertices[i] for i in region_vertices])
        clipped_polygon = polygon.intersection(boundary_polygon)
        if not clipped_polygon.is_empty:
            clipped_areas.append({
                'nflId': player_row.nflId,
                'frameId': player_row.frameId,
                'playId': player_row.playId,
                'position': player_row.position,
                'voronoiArea': clipped_polygon.area,
                'DisplayName': player_row.displayName_x
            })

    # Convert to DataFrame
    return pd.DataFrame(clipped_areas)


In [114]:
df  = compute_voronoi_areas(samplePlay)

,nflId,frameId,playId,position,voronoiArea,DisplayName
0,43329.0,1,2207,WR,10.642718,Sterling Shepard
1,43329.0,2,2207,WR,10.642718,Sterling Shepard
2,43329.0,3,2207,WR,0.347771,Sterling Shepard
3,43329.0,4,2207,WR,0.605634,Sterling Shepard
4,43329.0,5,2207,WR,0.878389,Sterling Shepard
...,...,...,...,...,...,...
1084,54577.0,214,2207,TE,1.733618,Daniel Bellinger
1085,54577.0,215,2207,TE,2.915803,Daniel Bellinger
1086,54577.0,216,2207,TE,2.839092,Daniel Bellinger
1087,54577.0,217,2207,TE,2.870183,Daniel Bellinger


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def plot_separation_animation_browser(separation_data, output_file="separation_animation.html", start_frame=10):
    # Filter data to start from the desired frame
    filtered_data = separation_data[separation_data['frameId'] >= start_frame]
    
    # Prepare the data: pivot for easy plotting
    pivot_data = filtered_data.pivot(index='frameId', columns='DisplayName', values='voronoiArea')
    players = pivot_data.columns

    # Set up the figure and axes
    fig, ax = plt.subplots()
    lines = {player: ax.plot([], [], label=f" {player}")[0] for player in players}

    # Configure plot limits and labels
    ax.set_xlim(filtered_data['frameId'].min(), filtered_data['frameId'].max())
    ax.set_ylim(0, 120)
    ax.set_xlabel("Frame")
    ax.set_ylabel("Voronoi Area (yards)")
    ax.set_title("Player Separation Over Time")
    ax.legend(loc='upper left')

    # Update function for animation
    def update(frame):
        for player in players:
            if frame in pivot_data.index:
                lines[player].set_data(pivot_data.index[:frame - start_frame], pivot_data[player].iloc[:frame - start_frame])
        return lines.values()

    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(pivot_data), blit=False, interval=50)

    # Save the animation as an HTML file
    from matplotlib.animation import HTMLWriter
    writer = HTMLWriter()
    ani.save(output_file, writer=writer)

    print(f"Animation saved as {output_file}")

    plt.close(fig)  




In [ ]:
plot_separation_animation_browser(df, start_frame=140)

Animation saved as separation_animation.html


: 